# 02. Tratamendo dos dados

Neste _notebook_ vou realizar a união dos dois datasets que eu extraí.

## Inicialização

Carregando as bibliotecas e as bases de dados que serão utilizadas nesse notebook.

In [15]:
import locale

import numpy as np
import pandas as pd

from operator import index
from datetime import datetime

In [16]:
# dataframe de indices socioeconomicos

df_se = pd.read_excel("./HDR21-22_Statistical_Annex_HDI_Table.xlsx")
df_feriados = pd.read_csv("./feriados.csv")

In [17]:
df_feriados.head(10)

,país,data,nome,tipo
0,afghanistan,15 de Fev,Liberation Day,Public Holiday
1,afghanistan,20 de Mar,March Equinox,Season
2,afghanistan,21 de Mar,Nauruz,Observance
3,afghanistan,2 de Abr,First Day of Ramadan,Public Holiday
4,afghanistan,28 de Abr,Afghan Victory Day,Public Holiday
5,afghanistan,1 de Mai,Eid al-Fitr,Public Holiday
6,afghanistan,1 de Mai,Labor Day,Public Holiday
7,afghanistan,2 de Mai,Eid al-Fitr Holiday,Public Holiday
8,afghanistan,3 de Mai,Eid al-Fitr Holiday,Public Holiday
9,afghanistan,21 de Jun,June Solstice,Season


dentre as variáveis presentes no dataframe relacionado aos feriados dos países temos:

- **país**: País onde aquele feriado ocorre;

- **data**: Essa variável pode ser um pouco enganosa, pois em alguns feriados ocorrem de acordo com um dia específico da semana combinado com algumas condições, como por exemplo a Páscoa Cristã, que é celebrada no primeiro domingo após a primeira lua cheia que ocorre depois do equinócio de primavera/outono ([fonte: Mundo Educação](https://mundoeducacao.uol.com.br/pascoa/data-pascoa.htm));

- **nome**: Nome do feriado que é celebrado no dia;

- **tipo**: Os tipos de feriado presentes no dataframe são diversos, como feriados relacionados a estação (Solstícios, Equinócios), feriados nacionais (aniversário do imperador do Japão, dias da independência ou datas comemorativas específicas), feriados apenas para trabalhadores do setor público (dia do setor público em Costa Rica), feriados estaduais... 

In [18]:
df_se.head(10)

,HDI rank,Country,Human Development Index (HDI),Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita,GNI per capita rank minus HDI rank,HDI rank.1
0,1,Switzerland,0.962,83.9872,16.500299,13.859660,66933.00454,5,3
1,2,Norway,0.961,83.2339,18.185200,13.003630,64660.10622,6,1
2,3,Iceland,0.959,82.6782,19.163059,13.767170,55782.04981,11,2
3,4,"Hong Kong, China (SAR)",0.952,85.4734,17.278170,12.226210,62606.84540,6,4
4,5,Australia,0.951,84.5265,21.054590,12.726820,49238.43335,18,5
5,6,Denmark,0.948,81.3753,18.714800,12.960490,60364.78595,6,5
6,7,Sweden,0.947,82.9833,19.418530,12.609720,54489.37401,9,9
7,8,Ireland,0.945,81.9976,18.945221,11.582223,76168.98443,-3,8
8,9,Germany,0.942,80.6301,17.010139,14.090967,54534.21682,6,7
9,10,Netherlands,0.941,81.6873,18.693165,12.581630,55979.41100,3,10


No banco de dados acima, temos as seguintes variáveis:

- **HDI rank** : Ranking atual de acordo com o IDH do País;

- **Country** : País referente aos índices indicados;

- **Human Development Index (HDI)** : valor do IDH recente (2020 - 2022)

    (para saber mais sobre o cálculo do IDH [clique aqui](https://educacao.uol.com.br/disciplinas/geografia/indice-de-desenvolvimento-humano-idh-mede-nivel-de-qualidade-de-vida.htm#:~:text=Como%20funciona%20o%20IDH,o%20desenvolvimento%20é%20considerado%20alto.))

- **Life expectancy at birth** : Expectativa de vida;

- **Expected years of schooling** : número de anos de escolaridade que uma criança/adolescencia passa para concluir toda sua educação;

- **Mean years of schooling** : Número médio de anos que um cidadão estuda;

- **Gross national income (GNI) per capita** : é o valor do PiB acrescido dos rendimentos líquidos provenientes do exterior através do pagamento de empregados 

As últimas duas colunas serão excluídas por não serem de interesse a este estudo.


## Tratamento dos dados

Vamos dividir essa etapa em 3 partes:
- tratamento do primeiro dataframe;
- tratamento do segundo dataframe;
- Agregação dos dataframes;

Iniciando pelo que aparenta ser o mais simples, o tratamento dos dados referentes aos feriados de cada país. Como o objetivo do projeto é analisar o impacto de feriados que realmente [MODIFICAR] TIRAM DIAS LETIVOS/DÃO DIAS DE FOLGA e os índices socioêconomicos, vamos realizar um agrupamento pelo tipo de feriado, deixando assim o trabalho de ciência de dados mais simples resumindo nossos dados

In [19]:
# Analizando os tipos de feriado
df_feriados['tipo'].value_counts()

Public Holiday                1964
Observance                    1208
National holiday              1011
Season                         920
United Nations observance      177
                              ... 
Orthodox, Optional holiday       1
Working day (replacement)        1
Academic Holiday                 1
Public Sector                    1
Prescribed Day of Rest           1
Name: tipo, Length: 123, dtype: int64

Existem 123 tipos diferentes de feriado, vamos analisar os mais recorrentes e filtrar os que possuem uma observação única para assim realizar uma análise completa.

Dentre os feriados que [dão folga] em dia letivo segundo o site no qual foram extraídos os dados, estão listados como:

- Public Holiday
- National Holiday
- Government Holiday
- State Holiday	
- State Legal Holiday	

Para realizar uma separação desses tipos de feriado dos demais (como apenas sazonais ou comemorativos nos quais os negócios funcionam normalmente), vamos agrupá-los em uma nova coluna no dataset.

In [20]:
# update: os valores na coluna "tipo" estão despadronizados e por isso não estão
# filtrando de forma adequada, vou padronizar colocando todos os valores em formato minusculo

df_feriados['tipo'] = [i.lower() for i in  df_feriados['tipo']]

# lista de feriados que realmente ''''Realmente tiram dia de folga'''''
lista_nfac = ['public holiday',
        'national holiday',
        'government holiday',
        'state holiday',
        'state legal holiday']

# lista auxiliar para indicar feriados
aux = []

# buscando 'matches' entre a lista de feriados e a coluna que possue todos os tipos de feirado
for s in df_feriados['tipo']:
        if any(xs in s for xs in lista_nfac):
                aux.append(1)
        else:
                aux.append(0)

# adicionando a coluna indicadora de feriados no dataframe
df_feriados['indicadora'] = aux

Vamos agora agrupar os dados por país, adicionando as colunas com o número total de feriados e o número de feriados que dão folga e adicionar essas informações ao que chamaremos de "df_final"

In [21]:
df_final = pd.DataFrame({"País": df_feriados.groupby('país').count().index.tolist(),
        "Feriados total" : df_feriados.groupby('país').count()["indicadora"],
        "Feriados folga" :  df_feriados.groupby('país').sum()["indicadora"]})


C:\Users\yurir\AppData\Local\Temp\ipykernel_10840\2722691182.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  "Feriados folga" :  df_feriados.groupby('país').sum()["indicadora"]})


In [22]:
df_final = df_final.reset_index()

# a coluna país se encopntra duplicada após o reset de index, vamos remover uma delas
df_final= df_final.drop(columns=['país'])

# visualizando como se encontra o dataframe até o momento
df_final


,País,Feriados total,Feriados folga
0,afghanistan,22,17
1,albania,39,21
2,algeria,16,12
3,american-samoa,17,13
4,andorra,29,14
...,...,...,...
227,vietnam,30,17
228,wallis-and-futuna,18,11
229,yemen,27,16
230,zambia,25,18


Vamos agora unir as informações relevantes presentes no banco de dados da unesco referente a valores socioeconômicos de cada país;

In [23]:
# Padronizando a coluna voltadas ao nome de país
df_se['Country'] = [i.lower() for i in df_se['Country']]
df_se['Country'] = [i.replace(" ", "-") for i in df_se['Country']]

# renomeando a coluna Country
df_se['País'] = df_se['Country']
df_se= df_se.drop(columns=['Country'])

In [24]:
# unindo os dataframes
df_final = pd.merge(df_final, df_se)

# filtrando apenas as colunas que serão usadas nesse estudo
df_final = df_final.drop(columns=['HDI rank.1', 'GNI per capita rank minus HDI rank'])

# arredondando alguns valores para melhor visualização
df_final['Life expectancy at birth'] = [round(i, 1) for i in df_final['Life expectancy at birth']]
df_final['Expected years of schooling'] = [round(i, 3) for i in df_final['Expected years of schooling']]
df_final['Mean years of schooling'] = [round(i, 3)for i in df_final['Mean years of schooling']]
df_final['Gross national income (GNI) per capita'] = [round(i, 1) for i in df_final['Gross national income (GNI) per capita']]

# visualizando o dataframe final
df_final

,País,Feriados total,Feriados folga,HDI rank,Human Development Index (HDI),Life expectancy at birth,Expected years of schooling,Mean years of schooling,Gross national income (GNI) per capita
0,afghanistan,22,17,180,0.478,62.0,10.264,2.985,1824.2
1,albania,39,21,67,0.796,76.5,14.448,11.286,14131.1
2,algeria,16,12,91,0.745,76.4,14.627,8.069,10800.2
3,andorra,29,14,40,0.858,80.4,13.300,10.555,51166.6
4,angola,18,14,148,0.586,61.6,12.172,5.417,5465.6
...,...,...,...,...,...,...,...,...,...
158,uzbekistan,27,19,101,0.727,70.9,12.477,11.896,7916.8
159,vanuatu,21,17,140,0.607,70.4,11.535,7.065,3085.4
160,yemen,27,16,183,0.455,63.8,9.099,3.200,1314.3
161,zambia,25,18,154,0.565,61.2,10.929,7.187,3217.8


In [25]:
#update: vamos renomear as colunas para facilitar o acesso das variáveis

df_final.rename(columns = {'HDI rank':'idh_rank', 
                           df_final.columns[4] :'idh',
                          'Life expectancy at birth' : 'expec_vida',
                          'Gross national income (GNI) per capita': 'GNI_percapita'}, inplace = True)

In [52]:
# Update: Adicionando continentes
# existe uma biblioteca chamada pycountry-convert que faz exatamente o que
# precisamos

# vou criar uma função para realizar essa tarefa

import pycountry_convert as pc

def mudanome(nomepais):
    
    # Padronizando para funcionamento integral da biblioteca

    # regra geral 
    nomepais = nomepais.replace("-", " ")

    # Para casos de país que possuem "and" no nome, capitalizaremos tudo com excessão da conjunção
    e = ['and', 'the']
    nomepais = ' '.join([word.capitalize() if word not in e else word for word in nomepais.split()])
    

    # países com nomes específicos na biblioteca
    nomepais = nomepais.replace("Guinea Bissau", "Guinea-Bissau")
    nomepais = nomepais.replace("Timor Leste", "Timor-Leste")
    print(nomepais)

    # muda o nome 
    try:
        country_code = pc.country_name_to_country_alpha2(nomepais, cn_name_format="default")
        continent_name = pc.country_alpha2_to_continent_code(country_code)
        return continent_name
    except ValueError as e:
         if str(e) == "Invalid Country Alpha-2 code: 'TL'":
            return country_code 



In [53]:
teste = ['Timor-Leste']

for i in teste:
    print(mudanome(i))


Timor-Leste


KeyError: "Invalid Country Alpha-2 code: 'TL'"

In [ ]:
df_final

,País,Feriados total,Feriados folga,idh_rank,idh,expec_vida,Expected years of schooling,Mean years of schooling,GNI_percapita
0,afghanistan,22,17,180,0.478,62.0,10.264,2.985,1824.2
1,albania,39,21,67,0.796,76.5,14.448,11.286,14131.1
2,algeria,16,12,91,0.745,76.4,14.627,8.069,10800.2
3,andorra,29,14,40,0.858,80.4,13.300,10.555,51166.6
4,angola,18,14,148,0.586,61.6,12.172,5.417,5465.6
...,...,...,...,...,...,...,...,...,...
158,uzbekistan,27,19,101,0.727,70.9,12.477,11.896,7916.8
159,vanuatu,21,17,140,0.607,70.4,11.535,7.065,3085.4
160,yemen,27,16,183,0.455,63.8,9.099,3.200,1314.3
161,zambia,25,18,154,0.565,61.2,10.929,7.187,3217.8


Agora possuimos um dataframe com 163 países e 9 colunas de informação. Vamos extrair esse banco de dados em formato de .csv e passar para os finalmentes: Estudos de correlação entre as colunas de feriados e um dashboard bem bonito para resumir nosso trabalho.

In [ ]:
df_final.to_csv("./final.csv", index = False)